In [8]:
from LDistanceModel import ldistance

# Example input strings
str1 = "kitten"
str2 = "sitting"

# Function call to calculate Levenshtein distance
distance = ldistance(str1, str2)

# Print the result
print("Levenshtein Distance:", distance)
import json
print("Json:", json.dumps(distance))

data = distance

Levenshtein Distance: [('Substitute', 'k', 's'), ('Nothing', 'i'), ('Nothing', 't'), ('Nothing', 't'), ('Substitute', 'e', 'i'), ('Nothing', 'n'), ('Insert', 'g')]
Json: [["Substitute", "k", "s"], ["Nothing", "i"], ["Nothing", "t"], ["Nothing", "t"], ["Substitute", "e", "i"], ["Nothing", "n"], ["Insert", "g"]]


In [9]:
str1 = "kitten"
str2 = "sitting"
unique_letters = set()
unique_letters.update(str1, str2)

unique_letters

{'e', 'g', 'i', 'k', 'n', 's', 't'}

In [10]:
embedding_dim = 4

from LDistanceModel import SetEmbedding

embedding = SetEmbedding(set_of_things=unique_letters, embedding_dim=embedding_dim)

embedding('g')

tensor([[-0.3236,  0.5717, -0.7189,  1.7287]])

In [11]:
data

[('Substitute', 'k', 's'),
 ('Nothing', 'i'),
 ('Nothing', 't'),
 ('Nothing', 't'),
 ('Substitute', 'e', 'i'),
 ('Nothing', 'n'),
 ('Insert', 'g')]

In [12]:
from LDistanceModel import operation_encoder

operation_encoder(data, embedding)

tensor([[ 0.5000,  0.0000, -0.5366,  0.3165, -0.0275, -1.7936],
        [ 0.0000,  0.5000, -0.1369,  0.0376,  2.0246, -0.4295],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.4625,  0.5968, -0.5838, -0.4736],
        [ 0.0000,  0.0000,  1.4625,  0.5968, -0.5838, -0.4736],
        [ 0.5000,  0.0000,  0.2372,  1.2232,  1.1071,  3.0175],
        [ 0.0000,  0.5000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.6138, -0.6678, -0.8481, -1.6216],
        [ 0.0000,  1.0000, -0.3236,  0.5717, -0.7189,  1.7287]])

In [13]:
string_pairs = [
    ("hello", "hallo"),
    ("cat", "bat"),
    ("python", "java"),
    ("same", "same"),
    ("a", "abc")
]

all_characters_in_pairs = set()
for pair in string_pairs:
    for word in pair:
        all_characters_in_pairs.update(word)

string_pair_embedder = SetEmbedding(all_characters_in_pairs, 5)

from LDistanceModel import create_batch_with_masks
create_batch_with_masks(string_pairs, string_pair_embedder)

100%|██████████| 5/5 [00:00<00:00, 5009.92it/s]


(tensor([[[ 0.0000,  0.0000,  0.7664,  0.7679,  0.6648, -0.3996, -0.0641],
          [ 0.5000,  0.0000, -0.8758,  0.8610,  0.5925, -1.7178, -0.2086],
          [ 0.0000,  0.5000, -1.1649, -0.3646,  0.4869,  2.2650, -1.7304],
          [ 0.0000,  0.0000,  1.4065, -2.2335,  1.0291, -0.6604, -0.8419],
          [ 0.0000,  0.0000,  1.4065, -2.2335,  1.0291, -0.6604, -0.8419],
          [ 0.0000,  0.0000,  1.2469, -0.2773,  1.8645,  0.8814, -2.0055],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],
 
         [[ 0.5000,  0.0000, -1.2388, -0.8532, -0.5534,  0.1772,  0.1094],
          [ 0.0000,  0.5000, -0.6214, -0.2040,  0.4633,  1.3502, -0.2752],
          [ 0.0000,  0.0000, -1.1649, -0.3646,  0.4869,  2.2650, -1.7304],
          [ 0.0000,  0.0000, -0.0952,  0.3766, -0.7306,  1.7517,  0.1024],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0

In [14]:
import torch

bruh = torch.tensor([1,2,3])
yeet = torch.tensor([1,2,3])

torch.stack((bruh, yeet))[:, 2]

[[1,2,3],[1,2,3]][:, 2]

<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\lsm0147\AppData\Local\Temp\ipykernel_26932\2517620060.py:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  [[1,2,3],[1,2,3]][:, 2]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
import graphviz

# Create a graphviz Digraph object
dot = graphviz.Digraph()

# Function to add nodes and edges to the graph in reverse order, skipping nodes with operation 'Nothing'
def add_nodes_edges_reverse_skip_nothing(data, parent=None):
    if data['operation'][0] == 'Nothing':
        if data['history']:
            add_nodes_edges_reverse_skip_nothing(data['history'], parent)
        return
    
    node_id = str(id(data))
    label = f"Distance: {data['distance']}\nOperation: {data['operation']}"
    dot.node(node_id, label)
    
    if parent:
        dot.edge(node_id, parent)
    
    if data['history']:
        add_nodes_edges_reverse_skip_nothing(data['history'], node_id)

# Add nodes and edges starting from the root
add_nodes_edges_reverse_skip_nothing(data)

# Save the graph to a file
dot.save('graph_reverse_skip_nothing_new.dot')

# Render the graph to a PNG image file
dot.render('graph_reverse_skip_nothing_new', format='png')

print("Graph has been successfully created and saved as graph_reverse_skip_nothing_new.png.")

In [15]:
# Function to filter words based on given criteria
def filter_words(words):
    filtered_words = []
    for word in words:
        if len(word) == 5 and word[0] != 'c' and word[1] != 'r' and word[3] != 'n' and 'a' not in word and 'e' not in word:
            if 'c' in word and 'r' in word and 'n' in word:
                filtered_words.append(word)

            filtered_words.append(word)
    return filtered_words

# Read the content of the file
with open('data/english.txt', 'r') as f:
    words = f.read().splitlines()

# Filter the words
filtered_words = filter_words(words)

# Print the filtered words
for word in filtered_words:
    print(word)

bhili
bhoot
bhuts
bib's
bibbs
bibio
bichy
bid's
biddy
bidis
bidri
biffs
biffy
bifid
biggs
biggy
bight
bigly
bigos
bigot
bijou
bikol
bilbo
bilby
bilch
bilgy
bilic
bilin
bilio
bilks
bills
billy
bilsh
bimbo
bin's
bindi
binds
bingo
bingy
binit
bints
biogs
bipod
birch
birds
birdy
birks
birls
biron
biros
birrs
birsy
birth
bisks
bison
bisti
bit's
bitch
bitis
bitsy
bitts
bitty
bixin
blick
blimp
blimy
blips
bliss
blitz
blizz
blobs
bloch
block
blocs
blogs
blood
bloom
bloop
blots
blout
blown
blows
blowy
blubs
bluff
blurb
blurs
blurt
blush
blyth
bob's
bobby
bocci
bocks
bocoy
bodhi
boffo
boffs
bog's
boggs
boggy
bogum
bogus
bohor
bohos
boiko
boils
boily
boist
bokom
boldo
bolds
boldu
bolis
bolls
bolly
bolos
bolti
bolts
bolus
bombo
bombs
bonbo
bonds
bongo
bongs
bonks
bonos
bonum
bonus
boobs
booby
boody
boogy
books
booky
booly
booms
boomy
boors
boort
boost
boosy
booth
boots
booty
boozy
borgh
boric
boris
borks
boron
borts
borty
bortz
boryl
bosch
bosks
bosky
bosom
boson
bossy
bosun
botch
bothy
botts
boug